We now want to take a point cloud generated from the plant surface mapping program, as well as an input model mesh, and fit the model within the plant.

We expect that the user wants to keep their model upright, but doesn't care about the rotation of their model about the vertical axis. The shape of the model must remain the same.

Therefore, valid transformations for fitting the model consist of even scaling, translations, and rotations around the z-axis.

In [ ]:
import pyvista as pv
import numpy as np
import 

Import the models

In [76]:
# Import model
reader = pv.get_reader('cube.obj')
mesh = reader.read()

# mesh.plot()

# Import point cloud from plant scan
reader = pv.get_reader('point_cloud.vtk')
cloud = reader.read()

# cloud.plot()

First simple method which should be foolproof:
1. Find the barycenter of the model, and center that at 0,0.
2. Find the furthest model vertex from the 0,0.
3. Find the closest plant point to the 0,0.
4. Scale the model by the ratio of the result of (3) over (2)

In [77]:
# Find the barycenter of the model
centers = mesh.cell_centers().points
barycenter = np.mean(centers, axis=0)

# Move to 0,0
mesh = mesh.translate(-barycenter)

# Find vertices
verts = mesh.points
verts = np.unique(verts, axis=0)

# Furthest vertex from 0,0
max_model_vert = np.max(np.linalg.norm(verts, axis=1), axis=0)

# Closest plant point to 0,0
cloud_points = cloud.points
min_plant_point = np.min(np.linalg.norm(cloud_points, axis=1), axis=0)

# Scale
scale = min_plant_point / max_model_vert
mesh = mesh.scale(scale)

p = pv.Plotter()
p.add_mesh(mesh)
p.add_points(cloud)

Actor (0x15282430640)
  Center:                     (-0.28505569733994207, -0.1164562111300187, 0.0)
  Pickable:                   True
  Position:                   (0.0, 0.0, 0.0)
  Scale:                      (1.0, 1.0, 1.0)
  Visible:                    True
  X Bounds                    -1.160E+01, 1.103E+01
  Y Bounds                    -1.162E+01, 1.139E+01
  Z Bounds                    -1.014E+01, 1.014E+01
  User matrix:                Identity
  Has mapper:                 True

Property (0x15282430940)
  Ambient:                     0.0
  Ambient color:               Color(name='lightblue', hex='#add8e6ff', opacity=255)
  Anisotropy:                  0.0
  Color:                       Color(name='lightblue', hex='#add8e6ff', opacity=255)
  Culling:                     "none"
  Diffuse:                     1.0
  Diffuse color:               Color(name='lightblue', hex='#add8e6ff', opacity=255)
  Edge color:                  Color(name='black', hex='#000000ff', opacity=255)
  

To create a toolpath, we want to sweep the cutting tool up through the plant in vertical strips around the z-axis.

For each vertical strip, we compute the depth from the point map to the mesh below at many points along the path to figure out how deep we need to cut along the path.

The plant point cloud here is useful because although we can know what the final depth of the cut will be, we don't want to cut too deep at once, but in thinner layers. 
The depth between the plant surface and final shape will let us keep track of where to start the cut and how many passes we need to get to the needed depth.

Before the toolpath is made, we should make a data structure containing, for each angle and vertical height, the starting point and cut depth. When we generate the toolpath, we can iterate through in passes, moving the starting point in by the cut depth each time and subtracting from the cut depth counter. We are done cutting an area when its depth counter reaches 0.

In [93]:
angle_step = 10
height_step = 1
height_range = [-12, 12]

cut_list = np.zeros(shape=(360//angle_step, (height_range[1]-height_range[0])//height_step+1, 2), dtype=object) # angle, height, (start point, cut depth)

for ai, angle in enumerate(range(0, 360, angle_step)):
    for hi, height in enumerate(range(height_range[0], height_range[1]+1, height_step)):
        # First, find the starting point. To do this, and be extra careful about the points from the scan,
        # we can find a few points nearest in phi and z, and then pick the one furthest from the origin.

        close_points = [((0,0,0), np.inf)]*3
        phi = np.radians(angle)

        for point in cloud_points:
            # Compare our height and angle at the point's radial distance
            point_rho = np.linalg.norm(point[:2])
            comparison_point = (np.cos(phi)*point_rho, np.sin(phi)*point_rho, height)

            for i in range(len(close_points)):
                if np.linalg.norm(point - comparison_point) < close_points[i][1]:
                    close_points[i] = (point, np.linalg.norm(point - comparison_point))
                    break

        # Now take furthest rho
        close_points_rho = [np.linalg.norm(p[0][:2]) for p in close_points]
        start_rho = np.max(close_points_rho)
        start_point = (np.cos(phi)*start_rho, np.sin(phi)*start_rho, height)

        # Next, the depth of the cut is the distance from the starting point to the mesh along a radius in xy.
        # We will need to intersect a ray starting at the start point and going in the -rho direction with the mesh.
        ray_start = start_point
        ray_end = (0, 0, height)

        # If there is no intersection, we must have to cut all the way to the center, so the depth is the rho of the start point.
        # If there is an intersection, the depth is the rho distance from the start point to the intersection point.
        intersection = mesh.ray_trace(ray_start, ray_end)
        if len(intersection[0]) == 0:
            cut_depth = start_rho
        else:
            intersection_point = intersection[0][0]
            cut_depth = np.linalg.norm(np.array(start_point) - np.array(intersection_point))

        cut_list[ai, hi] = (start_point, cut_depth)

print(cut_list)

            

[[[(np.float64(6.307909107046044), np.float64(0.0), -12)
   np.float64(6.307909107046044)]
  [(np.float64(6.307909107046044), np.float64(0.0), -11)
   np.float64(6.307909107046044)]
  [(np.float64(6.596238406931054), np.float64(0.0), -10)
   np.float64(6.596238406931054)]
  ...
  [(np.float64(6.71102280325275), np.float64(0.0), 10)
   np.float64(6.71102280325275)]
  [(np.float64(6.35050537250276), np.float64(0.0), 11)
   np.float64(6.35050537250276)]
  [(np.float64(6.711909615248807), np.float64(0.0), 12)
   np.float64(6.711909615248807)]]

 [[(np.float64(6.513224262578746), np.float64(1.1484571688976286), -12)
   np.float64(6.613701245402367)]
  [(np.float64(6.513224262578746), np.float64(1.1484571688976286), -11)
   np.float64(6.613701245402367)]
  [(np.float64(6.513224262578746), np.float64(1.1484571688976286), -10)
   np.float64(6.613701245402367)]
  ...
  [(np.float64(6.678948860222632), np.float64(1.1776788868293), 10)
   np.float64(6.781982412094026)]
  [(np.float64(6.2540269263

Now, to generate the list of toolpath instructions, we just need to follow through each cut over and over until we are done cutting.

The cutter angle will always follow the line between the current and next cut along the vertical strip. To start, we set this first angle, enable the cutter,
and move into the depth of the first cut. Then we move the extruder and gantry along a line connecting this cut to the next, all the way until the final cut.
The final move will be from the final cut to the 0,0,h_last+h_step to take us out of the vertical strip. Then we will back off the extruder and rotate to the next vertical strip.

We will continue cutting these vertical strips all the way around the plant. After, we will start all over again, cutting the next pass.
We will be done when all cut depths are 0.